In [1]:
# ライブラリのインポート
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
# 警告を表示したくない場合は以下を記述
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
# MNISTをダウンロードして参照情報をオブジェクトに格納
mnist = input_data.read_data_sets("data/", one_hot=True)
test_images = mnist.test.images   # テスト用の画像を変数に格納
test_labels = mnist.test.labels   # テスト用の正解ラベルを変数に格納

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
# 学習率、エポック数、ミニバッチのサイズを設定
learning_rate = 0.001  # 学習率
training_epochs = 1500 # エポック数
batch_size = 100       # ミニバッチのサイズ

In [4]:
## データフローグラフ

# 入力層（ニューロン数＝784→1に相当）
with tf.name_scope("input"):
    # 入力データ（行数可変, 各データのサイズ＝784列）
    x = tf.placeholder(tf.float32, [None, 784])
    # 画像データの2次元配列「データ数×784」を
    # 「データ数×28×28×1」の4次元配列に変換
    out_img = tf.reshape(x,[-1,28,28,1])

# 第1層：畳み込み層（ニューロン数＝32）
with tf.name_scope("layer1_conv"):
    # 2次元フィルター（3×3を32枚）
    f1 = tf.Variable(
        tf.truncated_normal([3,         # フィルターの縦サイズ
                             3,         # フィルターの横サイズ
                             1,         # 画像のチャンネル数
                             32],       # フィルターの枚数
                            stddev=0.1))
    # 畳み込みを行うopノード
    conv1 = tf.nn.conv2d(out_img,       # 28×28の画像
                         f1,            # 2次元フィルター
                         strides=[1,    # 入力データを1ずつ順送り
                                  1,    # 縦方向のストライドは1
                                  1,    # 横方向のストライドは1
                                  1],   # チャンネルのストライドは1
                         padding='SAME' # パディングを行う
                        )
    # フィルターのバイアス
    b1 = tf.Variable(
        tf.constant(0.1,                # 0.1で初期化
                    shape=[32]          # フィルターの数だけ用意
                    ))
    # 活性化関数はReLU
    # 出力：(画像の枚数, 28（タテ）, 28（ヨコ）, 32（チャンネル）)
    out_conv1 = tf.nn.relu(conv1 + b1)

# Flatten
# (画像の枚数, 28（タテ）,28（ヨコ）,32（チャンネル）)を
# (画像の枚数, 28×28×32）の2階テンソルに変換
with tf.name_scope("flatten"):
    out_flat = tf.reshape(out_conv1,
                              [-1, 28*28*32] # (画像の枚数, 画像データ)
                             )

# 第2層（全結合層）：ニューロン数＝128
with tf.name_scope("layer5_binding"):
    # 重み(前層のニューロン数＝28×28×32, ニューロン数＝128列)
    w_comb1 = tf.Variable(tf.truncated_normal([28*28*32, 128], stddev=0.1))
    # バイアス(ニューロン数＝128列)
    b_comb1 = tf.Variable(tf.constant(0.1, shape=[128]))
    # 活性化関数:出力((前層の出力行数＝画像の枚数行, ニューロン数＝1024列)
    out_comb1 = tf.nn.relu(tf.matmul(out_flat, w_comb1) + b_comb1)

# 第3層（出力層）：ニューロン数＝10
with tf.name_scope("layer7_output"):
    # 重み(前層のニューロン数＝1024行,クラス数＝ニューロン数＝10列)
    w_comb2 = tf.Variable(tf.truncated_normal([128, 10], stddev=0.1))
    # バイアス(クラス数＝ニューロン数＝10列)
    b_comb2 = tf.Variable(tf.constant(0.1, shape=[10]))
    # 活性化関数:出力((前層の出力行数＝画像の枚数行, ニューロン数＝10列)
    out = tf.nn.softmax(tf.matmul(out_comb1, w_comb2) + b_comb2)

# 誤差関数（クロスエントロピー）
with tf.name_scope("loss"):
    # 正解ラベルを保持するプレースホルダー(行数可変, 列数は正解ラベルと同じ1)
    t = tf.placeholder(tf.float32, [None, 10])
    loss = tf.reduce_mean(
        -tf.reduce_sum(t*tf.log(out + 1e-5), axis=[1])
        )

# 訓練(ミニマイザー)
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(
        learning_rate).minimize(loss)

# 評価
with tf.name_scope("accuracy"):
    # (out＝t)の最大値を比較
    correct = tf.equal(tf.argmax(out,1), tf.argmax(t,1))
    # True(正解＝1)の平均を取る
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# 変数を初期化するopノード
init =tf.global_variables_initializer()

In [5]:
## セッション部

with tf.Session() as sess:
    sess.run(init)   # initノードを実行して変数を初期化

    for step in range(training_epochs):
        # ミニバッチを抽出
        train_images, train_labels = mnist.train.next_batch(batch_size)
        # 学習を実行
        sess.run(
            train_step,                    # ミニマイザーを実行
            feed_dict={x:train_images,     # 訓練データのミニバッチ
                       t:train_labels}     # 正解ラベルのミニバッチ
            )

        # 50回ごとにテストデータを使用して精度を出力
        step += 1
        if step % 50 == 0:
            acc_val = sess.run(
                accuracy,                  # クロスエントロピー誤差の平均オペレーション
                feed_dict={x:test_images,  # テストテータをセット
                           t:test_labels}  # テストデータの正解ラベル
            )
            print('Step %d: accuracy = %.2f' % (step, acc_val))

Step 50: accuracy = 0.81
Step 100: accuracy = 0.88
Step 150: accuracy = 0.90
Step 200: accuracy = 0.90
Step 250: accuracy = 0.92
Step 300: accuracy = 0.92
Step 350: accuracy = 0.92
Step 400: accuracy = 0.93
Step 450: accuracy = 0.94
Step 500: accuracy = 0.95
Step 550: accuracy = 0.95
Step 600: accuracy = 0.95
Step 650: accuracy = 0.96
Step 700: accuracy = 0.96
Step 750: accuracy = 0.96
Step 800: accuracy = 0.95
Step 850: accuracy = 0.96
Step 900: accuracy = 0.97
Step 950: accuracy = 0.97
Step 1000: accuracy = 0.97
Step 1050: accuracy = 0.97
Step 1100: accuracy = 0.97
Step 1150: accuracy = 0.97
Step 1200: accuracy = 0.97
Step 1250: accuracy = 0.98
Step 1300: accuracy = 0.98
Step 1350: accuracy = 0.98
Step 1400: accuracy = 0.98
Step 1450: accuracy = 0.98
Step 1500: accuracy = 0.98
